# 数据迭代

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_iterator.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_iterator.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/advance/dataset/iterator.ipynb)

原始数据集通过数据集加载接口读取到内存，再通过数据增强操作进行数据变换，最后得到的数据集对象有两种常规的数据迭代方法：

1. 创建`iterator`迭代器进行数据迭代。
2. 数据直接传入网络模型的Model接口（如`model.train`、`model.eval`等）进行迭代训练或推理。

## 创建迭代器

数据集对象通常可以创建两种不同的迭代器来遍历数据，分别为：

1. **元组迭代器**。创建元组迭代器的接口为`create_tuple_iterator`，通常用于`Model.train`内部使用，其迭代出来的数据可以直接用于训练。
2. **字典迭代器**。创建字典迭代器的接口为`create_dict_iterator`，自定义`train`训练模式下，用户可以根据字典中的`key`进行进一步的数据处理操作，再输入到网络中，使用较为灵活。

下面通过示例介绍两种迭代器的使用方式：

In [1]:
import mindspore.dataset as ds

# 数据集
np_data = [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]

# 加载数据
dataset = ds.NumpySlicesDataset(np_data, column_names=["data"], shuffle=False)

然后使用`create_tuple_iterator`或者`create_dict_iterator`创建数据迭代器。

In [2]:
# 创建元组迭代器
print("\n create tuple iterator")
for item in dataset.create_tuple_iterator():
    print("item:\n", item[0])

# 创建字典迭代器
print("\n create dict iterator")
for item in dataset.create_dict_iterator():
    print("item:\n", item["data"])

# 直接遍历数据集对象（等同于创建元组迭代器）
print("\n iterate dataset object directly")
for item in dataset:
    print("item:\n", item[0])

# 使用enumerate方式遍历（等同于创建元组迭代器）
print("\n iterate dataset using enumerate")
for index, item in enumerate(dataset):
    print("index: {}, item:\n {}".format(index, item[0]))


 create tuple iterator
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]

 create dict iterator
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]

 iterate dataset object directly
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]

 iterate dataset using enumerate
index: 0, item:
 [[1 2]
 [3 4]]
index: 1, item:
 [[5 6]
 [7 8]]


如果需要产生多个epoch的数据，可以相应地调整入参`num_epochs`的取值。相比于多次调用迭代器接口，直接设置epoch数可以提高数据迭代的性能。

In [3]:
# 创建元组迭代器产生2个epoch的数据
epoch = 2

iterator = dataset.create_tuple_iterator(num_epochs=epoch)

for i in range(epoch):
    print("epoch: ", i)
    for item in iterator:
        print("item:\n", item[0])

epoch:  0
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]
epoch:  1
item:
 [[1 2]
 [3 4]]
item:
 [[5 6]
 [7 8]]


迭代器默认输出的数据类型为`mindspore.Tensor`，如果希望得到`numpy.ndarray`类型的数据，可以设置入参`output_numpy=True`。

In [4]:
# 默认输出类型为mindspore.Tensor
for item in dataset.create_tuple_iterator():
    print("dtype: ", type(item[0]), "\nitem:\n", item[0])

# 设置输出类型为numpy.ndarray
for item in dataset.create_tuple_iterator(output_numpy=True):
    print("dtype: ", type(item[0]), "\nitem:\n", item[0])

dtype:  <class 'mindspore.common.tensor.Tensor'> 
item:
 [[1 2]
 [3 4]]
dtype:  <class 'mindspore.common.tensor.Tensor'> 
item:
 [[5 6]
 [7 8]]
dtype:  <class 'numpy.ndarray'> 
item:
 [[1 2]
 [3 4]]
dtype:  <class 'numpy.ndarray'> 
item:
 [[5 6]
 [7 8]]


下面我们通过一个拟合函数的场景，介绍在神经网络中如何使用数据迭代器，函数表达式为：$output = {x_0}\times1 + {x_1}\times2 + {x_2}\times3 + {x_7}\times8$。

In [5]:
import mindspore.dataset as ds
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import amp
from mindspore.common.initializer import Normal
import numpy as np

def func(x):
    """定义函数表达式"""
    result = []
    for sample in x:
        total = 0
        for i, e in enumerate(sample):
            total += (i+1) * e
        result.append(total)
    return result

class MyData:
    """自定义训练用数据集类"""
    def __init__(self):
        """初始化操作"""
        self.__data = np.array([[[1, 1, 1, 1, 1, 1, 1, 1]],
                                [[1, 1, 1, 1, 1, 1, 1, 0]],
                                [[1, 1, 1, 1, 1, 1, 0, 0]],
                                [[1, 1, 1, 1, 1, 0, 0, 0]],
                                [[1, 1, 1, 1, 0, 0, 0, 0]],
                                [[1, 1, 1, 0, 0, 0, 0, 0]],
                                [[1, 1, 0, 0, 0, 0, 0, 0]],
                                [[1, 0, 0, 0, 0, 0, 0, 0]]]).astype(np.float32)
        self.__label = np.array([func(x) for x in self.__data]).astype(np.float32)

    def __getitem__(self, index):
        """定义随即访问函数"""
        return self.__data[index], self.__label[index]

    def __len__(self):
        """定义获取数据集大小函数"""
        return len(self.__data)

class MyEvalData:
    """自定义验证用数据集类"""
    def __init__(self):
        """初始化操作"""
        self.__data = np.array([[[1, 1.1, 1, 1, 1, 1, 1, 1]],
                                [[1, 1.1, 1, 1, 1, 1, 1, 0]],
                                [[1, 1.1, 1, 1, 1, 1, 0, 0]],
                                [[1, 1.1, 1, 1, 1, 0, 0, 0]],
                                [[1, 1.1, 1, 1, 0, 0, 0, 0]],
                                [[1, 1.1, 1, 0, 0, 0, 0, 0]],
                                [[1, 1.1, 0, 0, 0, 0, 0, 0]],
                                [[1.1, 0, 0, 0, 0, 0, 0, 0]]]).astype(np.float32)
        self.__label = np.array([func(x) for x in self.__data]).astype(np.float32)

    def __getitem__(self, index):
        """定义随即访问函数"""
        return self.__data[index], self.__label[index]

    def __len__(self):
        """定义获取数据集大小函数"""
        return len(self.__data)


class MyNet(nn.Cell):
    """自定义网络"""
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc = nn.Dense(8, 1, weight_init=Normal(0.02))

    def construct(self, x):
        x = self.fc(x)
        return x

if __name__ == "__main__":
    print("--------- Train ---------")
    # 训练过程
    dataset_generator = MyData()
    dataset = ds.GeneratorDataset(dataset_generator, ["data", "label"], shuffle=False)
    net = MyNet()  # 定义网络
    optimizer = nn.Momentum(net.trainable_params(), 0.01, 0.9)  # 定义优化器
    loss = nn.MSELoss(reduction="mean")  # 定义损失函数
    train_network = amp.build_train_network(net, optimizer, loss, level="O3", loss_scale_manager=None)
    for epoch in range(20):  # 训练20次
        for step, item in enumerate(dataset.create_dict_iterator()):
            data = item["data"]
            label = item["label"]
            loss = train_network(data, label)
        print("epoch:{}, loss: {}".format(epoch, loss))

    print("--------- Eval ---------")
    # 推理过程
    eval_data = MyEvalData()
    for item in eval_data:
        predict = net(Tensor(item[0]))[0]
        print("predict: {}, label: {}".format(predict, item[1]))

--------- Train ---------
epoch:0, loss: 84.84137
epoch:1, loss: 11.769456
epoch:2, loss: 0.22432804
epoch:3, loss: 1.4706595
epoch:4, loss: 0.39552307
epoch:5, loss: 0.8743346
epoch:6, loss: 0.020189524
epoch:7, loss: 0.7218361
epoch:8, loss: 0.049575806
epoch:9, loss: 0.05044937
epoch:10, loss: 0.01478219
epoch:11, loss: 0.00044083595
epoch:12, loss: 0.009922028
epoch:13, loss: 0.0014505386
epoch:14, loss: 0.0048754215
epoch:15, loss: 0.0020179749
epoch:16, loss: 0.0010228753
epoch:17, loss: 0.00077462196
epoch:18, loss: 0.00025963783
epoch:19, loss: 0.00026756525
--------- Eval ---------
predict: [36.06], label: [36.2]
predict: [28.4], label: [28.2]
predict: [21.14], label: [21.2]
predict: [15.16], label: [15.2]
predict: [10.21], label: [10.2]
predict: [6.21], label: [6.2]
predict: [3.178], label: [3.2]
predict: [1.065], label: [1.1]


从上面的打印结果可以看出，随着训练次数逐渐增多，损失值趋于收敛，推理结果也较为准确。

> 更多关于数据迭代器的使用说明，请参考[create_tuple_iterator](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/dataset/mindspore.dataset.NumpySlicesDataset.html#mindspore.dataset.NumpySlicesDataset.create_tuple_iterator) 和[create_dict_iterator](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/dataset/mindspore.dataset.NumpySlicesDataset.html#mindspore.dataset.NumpySlicesDataset.create_dict_iterator)的API文档。

## 数据迭代训练

数据集对象创建后，可通过传入`Model`接口，由接口内部进行数据迭代，并送入网络执行训练或推理。实例代码如下：

In [6]:
import numpy as np
from mindspore import ms_function
from mindspore import nn, Model
import mindspore.dataset as ds
import mindspore.ops as ops

def create_dataset():
    """创建自定义数据集"""
    np_data = [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]
    np_data = np.array(np_data, dtype=np.float16)
    dataset = ds.NumpySlicesDataset(np_data, column_names=["data"], shuffle=False)
    return dataset

class Net(nn.Cell):
    """创建一个神经网络"""
    def __init__(self):
        super(Net, self).__init__()
        self.relu = ops.ReLU()
        self.print = ops.Print()

    @ms_function
    def construct(self, x):
        self.print(x)
        return self.relu(x)

if __name__ == "__main__":
    dataset = create_dataset()

    network = Net()
    model = Model(network)

    # 数据集传入model中，train接口进行数据迭代处理
    model.train(epoch=1, train_dataset=dataset)